In [2]:
import pandas as pd
import numpy as np
import warnings

In [3]:
#train 및 test 데이터 읽어 하나의 데이터로 합치기
df_train = pd.read_csv("../data/train.csv")
df_test = pd.read_csv("../data/test.csv")

df_all = pd.concat([df_train, df_test])
df_all.reset_index(drop = True, inplace = True)

print(df_train.shape, df_test.shape, df_all.shape)
df_all["SalePrice"].isnull().sum()

(1460, 81) (1459, 80) (2919, 81)


1459

In [4]:
# rename 칼럼 및 categoric의 class 
rename_dict = {"1stFlrSF":"FirstFlrSF", "2ndFlrSF":"SecondFlrSF", "3SsnPorch":"ThreeSsnPorch"}
df_all.rename(index = str, columns = rename_dict, inplace = True)

df_all["MSZoning"].replace({"C (all)":"all"}, inplace=True)
df_all["Exterior1st"].replace({"Wd Sdng":"WdSdng"}, inplace=True)
df_all["Exterior2nd"].replace({"Wd Sdng":"WdSdng"}, inplace=True)
df_all["Exterior2nd"].replace({"Wd Shng":"WdShng"}, inplace=True)
df_all["Exterior2nd"].replace({"Brk Cmn":"BrkCmn"}, inplace=True)
df_all["HouseStyle"].replace({"1.5Fin":"1_5Fin"}, inplace=True)
df_all["HouseStyle"].replace({"1.5Unf":"1_5Unf"}, inplace=True)
df_all["HouseStyle"].replace({"2.5Fin":"2_5Fin"}, inplace=True)
df_all["HouseStyle"].replace({"2.5Unf":"2_5Unf"}, inplace=True)
df_all["RoofMatl"].replace({"Tar&Grv":"TarandGrv"}, inplace=True)

In [5]:
# missing value 처리
df_all.loc["1379","Electrical"] = "SBrkr"

df_all["MasVnrType"].fillna("None", inplace = True)
df_all["MasVnrArea"].fillna(0, inplace = True)

df_all["BsmtQual"].fillna("None", inplace = True)
df_all["BsmtCond"].fillna("None", inplace = True)
df_all["BsmtFinType1"].fillna("None", inplace = True)
df_all["BsmtFinType2"].fillna("None", inplace = True)
df_all["BsmtExposure"].fillna("None", inplace = True)
df_all.loc["332","BsmtFinType2"] = "Rec"
df_all.loc["948","BsmtExposure"] = "No"

df_all["PoolQC"].fillna("None", inplace = True)
df_all["MiscFeature"].fillna("None", inplace = True)
df_all["Alley"].fillna("None", inplace = True)
df_all["Fence"].fillna("None", inplace = True)
df_all["FireplaceQu"].fillna("None", inplace = True)
df_all["GarageYrBlt"].fillna(0, inplace = True)
df_all["GarageType"].fillna("None", inplace = True)
df_all["GarageFinish"].fillna("None", inplace = True)
df_all["GarageQual"].fillna("None", inplace = True)
df_all["GarageCond"].fillna("None", inplace = True)

##!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 다시 처리
#numeric은 0, categoric은 None으로 missing value 처리
for c in df_all.columns:
    if df_all[c].dtype == "object":
        mode = df_all[c].mode().values[0]
        df_all[c].fillna(mode, inplace = True)
    else:
        df_all[c].fillna(0, inplace = True)

df_all.isnull().sum().sort_values(ascending = False).sum()

0

In [6]:
# log, **2, **3 생성
nuTocaList = ["MSSubClass", "MoSold", "YrSold", "OverallQual", "OverallCond"]
df_all[nuTocaList] = df_all[nuTocaList].astype("str")

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_list = list(df_all.select_dtypes(include=numerics).columns)

print(len(numeric_list))

#log
for c in numeric_list:
    if(c == "Id"):
        continue
    
    if(c == "SalePrice"):
        df_all[c] = np.log1p(df_all[c].values)
        df_all.rename(index = str, columns = {"SalePrice":"SalePriceLog"}, inplace = True)
        continue
        
    df_all[c+"Log"] = np.log1p(df_all[c].values)
    
# ** 2
for c in numeric_list:
    if(c == "Id" or c == "SalePrice"):
        continue
    
    df_all[c+"pow2"] = df_all[c].values ** 2

# ** 3
for c in numeric_list:
    if(c == "Id" or c == "SalePrice"):
        continue
    
    df_all[c+"pow3"] = df_all[c].values ** 3
    
df_all.shape

33


(2919, 174)

In [7]:
# get_dummies
df_all = pd.get_dummies(df_all)
print(df_all.shape)

df_train = df_all.iloc[:1460]
df_test = df_all.iloc[1460:]

print(df_train.shape, df_test.shape)

(2919, 444)
(1460, 444) (1459, 444)


In [8]:
#make csv file
df_train.to_csv("../data/train_dummy.csv", index = False)
df_test.to_csv( "../data/test_dummy.csv", index = False)

In [9]:
df_train

,FirstFlrSF,SecondFlrSF,ThreeSsnPorch,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,EnclosedPorch,...,SaleType_WD,Street_Grvl,Street_Pave,Utilities_AllPub,Utilities_NoSeWa,YrSold_2006,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010
0,856,854,0,3,706.0,0.0,1.0,0.0,150.0,0,...,1,0,1,1,0,0,0,1,0,0
1,1262,0,0,3,978.0,0.0,0.0,1.0,284.0,0,...,1,0,1,1,0,0,1,0,0,0
2,920,866,0,3,486.0,0.0,1.0,0.0,434.0,0,...,1,0,1,1,0,0,0,1,0,0
3,961,756,0,3,216.0,0.0,1.0,0.0,540.0,272,...,1,0,1,1,0,1,0,0,0,0
4,1145,1053,0,4,655.0,0.0,1.0,0.0,490.0,0,...,1,0,1,1,0,0,0,1,0,0
5,796,566,320,1,732.0,0.0,1.0,0.0,64.0,0,...,1,0,1,1,0,0,0,0,1,0
6,1694,0,0,3,1369.0,0.0,1.0,0.0,317.0,0,...,1,0,1,1,0,0,1,0,0,0
7,1107,983,0,3,859.0,32.0,1.0,0.0,216.0,228,...,1,0,1,1,0,0,0,0,1,0
8,1022,752,0,2,0.0,0.0,0.0,0.0,952.0,205,...,1,0,1,1,0,0,0,1,0,0
9,1077,0,0,2,851.0,0.0,1.0,0.0,140.0,0,...,1,0,1,1,0,0,0,1,0,0
